In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [19]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('datasets/digit_kaggle/train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data_arr = np.array(data)

In [16]:
data_arr

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [7, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0]])

In [42]:
y_data = data_arr[:,0]

In [43]:
X_data = data_arr[:,1:]

In [82]:
# Helper functions
def leaky_relu(z):
    return tf.maximum(0.01*z,z)

In [96]:
# Build the graph
tf.reset_default_graph()

n_inputs = 784
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

# Placeholders for X and y
X = tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')
y = tf.placeholder(tf.int32,shape=(None),name='y')

he_init = tf.variance_scaling_initializer()

with tf.name_scope("dnn"):
    
    layer1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.elu,
                             kernel_initializer=he_init, name='layer1' )
    layer2 = tf.layers.dense(layer1,n_hidden2,activation=tf.nn.elu,
                             kernel_initializer=he_init, name='layer2')
    logits = tf.layers.dense(layer2,10,name='logits')

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy,name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()


In [97]:
X_train, X_test = X_data[:35000], X_data[35000:]
y_train, y_test = y_data[:35000], y_data[35000:]
X_train = X_train.astype(np.float32).reshape(-1,28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1,28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [98]:
def shuffle_batch(X,y,batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx,n_batches):
        X_batch, y_batch = X[batch_idx],y[batch_idx]
        yield X_batch, y_batch

In [99]:
n_epochs = 100
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch,y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X:X_batch,y:y_batch})
        if epoch % 5 == 0:
            acc_batch = accuracy.eval(feed_dict={X:X_batch,y:y_batch})
            acc_valid = accuracy.eval(feed_dict={X:X_test,y:y_test})
            print(epoch, "Batch accuracy:{} --- Validation accuracy:{}".format(acc_batch,acc_valid))
                

0 Batch accuracy:0.9200000166893005 --- Validation accuracy:0.8811428546905518
5 Batch accuracy:0.8999999761581421 --- Validation accuracy:0.9194285869598389
10 Batch accuracy:0.9399999976158142 --- Validation accuracy:0.9315714240074158
15 Batch accuracy:0.9599999785423279 --- Validation accuracy:0.9404285550117493
20 Batch accuracy:0.9800000190734863 --- Validation accuracy:0.9475714564323425
25 Batch accuracy:0.9800000190734863 --- Validation accuracy:0.952571451663971
30 Batch accuracy:0.9800000190734863 --- Validation accuracy:0.9557142853736877
35 Batch accuracy:0.9399999976158142 --- Validation accuracy:0.9578571319580078
40 Batch accuracy:0.9599999785423279 --- Validation accuracy:0.9602857232093811
45 Batch accuracy:0.9399999976158142 --- Validation accuracy:0.9618571400642395
50 Batch accuracy:0.9800000190734863 --- Validation accuracy:0.9645714163780212
55 Batch accuracy:1.0 --- Validation accuracy:0.965571403503418
60 Batch accuracy:0.9599999785423279 --- Validation accurac

In [ ]:
# log
# -----------------

# -----------------
"""
n_epochs = 40
batch_size = 100
activation = relu
35 Batch accuracy:0.9900000095367432 --- Validation accuracy:0.9597142934799194
-------------------
n_epochs = 40
batch_size = 50
activation = relu
35 Batch accuracy:1.0 --- Validation accuracy:0.968999981880188
-------------------
n_epochs = 40
batch_size = 25
activation = relu
35 Batch accuracy:1.0 --- Validation accuracy:0.9731428623199463
-------------------
n_epochs = 40
batch_size = 10
activation = relu
35 Batch accuracy:1.0 --- Validation accuracy:0.9768571257591248
-------------------
n_epochs = 100
batch_size = 25
activation = relu
95 Batch accuracy:1.0 --- Validation accuracy:0.977142870426178
-------------------
n_epochs = 100
batch_size = 25
activation = leaky_relu
95 Batch accuracy:1.0 --- Validation accuracy:0.9747142791748047
-------------------
n_epochs = 100
batch_size = 50
activation = leaky_relu
95 Batch accuracy:1.0 --- Validation accuracy:0.9729999899864197
-------------------
n_epochs = 100
batch_size = 50
activation = elu
95 Batch accuracy:1.0 --- Validation accuracy:0.972000002861023
-------------------
"""